In [1]:
import cv2
import logging
import numpy
import typing
import math
from utils import (
    create_folder, 
    get_logger, 
    debounce, 
    throttle, 
    get_homography_martix, 
    select_points
)
from part1_interface import(
    gaussian_2d_convolution,
    convolution_2D,
)
logger: logging.Logger = get_logger(__name__)

In [2]:
image1 = cv2.imread('im011/im01.jpg')
image1 = cv2.cvtColor(src=image1, code=cv2.COLOR_BGR2GRAY)
image2 = cv2.imread('im011/im02.jpg')
image2 = cv2.cvtColor(src=image2, code=cv2.COLOR_BGR2GRAY)
image2 = cv2.resize(src=image2, dsize=(image1.shape[1], image1.shape[0]))

In [3]:
def get_key_points():
    key_points = []
    pass
# def get_key_points_decipher(image: numpy.ndarray, 
#                             num_features:int = 6, 
#                             sigma: float=1.52,

# )-> typing.Tuple[typing.List[typing.Any], 
#                  numpy.ndarray[typing.Any, 
#                                typing.Any]]:
def get_key_points_decipher(image: numpy.ndarray, 
                            num_features:int = 6, 
                            sigma: float=1.52,

):
    """
    :param image: input image
    :param num_features: the num of features to extract
    :param sigma: sigma of the first stack.
                  it's better to use √(1.6^2 - 0.5^2) which is the defualt value
    :
    """
    scale_factor = 1
    s = num_features  + 3
    o = max(int(numpy.log2(min(image.shape[0], image.shape[1]))) - 3, 3)
    k = 2 **(1/num_features)
    sigmas: typing.List[typing.List[float]] = []
    origin_images: typing.List[numpy.ndarray] = []
    resized_image = image
    size:int = int(sigma * 6 + 1)
    for O in range(o):
        row = []
        for S in range(s):
            value = (k ** S) * sigma * (1 << O)
            row.append(value)
        sigmas.append(row)
        new_width = int(image.shape[1] * scale_factor)
        new_height = int(image.shape[0] * scale_factor)
        scale_factor /= 2
        resized_image = cv2.resize(resized_image, (new_width, new_height))
        origin_images.append(resized_image)
    guassian_pyramid: typing.List[typing.List[numpy.ndarray]] = []
    for i in range(o):
        current_gussian: typing.List[numpy.ndarray]= []
        guassian_pyramid.append(current_gussian)
        for j in range(s):
            cur_sigma:float = sigmas[i][j]
            guassian_2d_kernel = gaussian_2d_convolution(size=size, sigma=cur_sigma)
            logger.info(f"origin_images[i].shape:{origin_images[i].shape}, size:{size}")
            guassian_result = convolution_2D(src=origin_images[i], kernel=guassian_2d_kernel)
            current_gussian.append(guassian_result)
    DoG: typing.List[typing.List[numpy.ndarray]] = []
    for O in range(o):
        DoG_row:typing.List[numpy.ndarray] = []
        DoG.append(DoG_row)
        for S in range(s - 1):
            diff_of_gaussian = guassian_pyramid[O][S + 1] - guassian_pyramid[O][S]
            DoG_row.append(diff_of_gaussian)
    # the value that I get from cv2.getContrastThreshold
    # contrast_threshold = 0.04
    # but according to lecture notes, it is 0.03
    contrast_threshold = 0.03
    # the value that I get from cv2.getEdgeThreshold
    # edge_threshold = 10.0
    # but according to lecture notes, it is 12.1
    edge_threshold = 12.1
    count = 0
    cur_neighbors: typing.List[typing.Union[int, float]] = []
    # tuple(O, S , i , j)
    row_extremas: typing.List[typing.Tuple[int, int, int, int]] = []
    for O in range(o):
        for S in range(1, s-1):
            threshold = (contrast_threshold /2) / (num_features * 255)
            cur_gussian_diff_image = DoG[O][S]
            pre_gussian_diff_image = DoG[O][S - 1]
            after_gussian_diff_image = DoG[O][S + 1]
            for i in range(1, cur_gussian_diff_image.shape[0] - 1):
                for j in range(1, cur_gussian_diff_image.shape[1] - 1):
                    # computing the Derivative for every dot is too slow
                    # filter the raw extremas by compare with the surrounding 26 points
                    cur_neighbors.clear()
                    cur_neighbors.extend(cur_gussian_diff_image[i: i +2, j-1: j + 2])
                    cur_neighbors.extend(pre_gussian_diff_image[i: i + 2, j -1 : j + 2])
                    cur_neighbors.extend(after_gussian_diff_image[i: i +2, j -1: j +2])
                    if cur_gussian_diff_image[i, j] > 0:
                        if(all( cur_gussian_diff_image[i, j] >= item_dot for item_dot in cur_neighbors)):
                            row_extremas.append((O, S, i, j))
                    elif cur_gussian_diff_image[i, j] < 0:
                        if(all( cur_gussian_diff_image[i, j] <= item_dot for item_dot in cur_neighbors)):
                            row_extremas.append((O, S, i, j))
                    else:
                        # cur_gussian_diff_image[i, j] == 0
                        if(all( cur_gussian_diff_image[i, j] <= item_dot for item_dot in cur_neighbors) or all( cur_gussian_diff_image[i, j] >= item_dot for item_dot in cur_neighbors)):
                            row_extremas.append((O, S, i, j))
    logger.info(f"row_extremas:{row_extremas}")
    filtered_and_corrected_extremas: typing.List[typing.Tuple[int, int, int, int, int]] = []
    for  O, S, i, j in row_extremas:
        cur_gussian_diff_image = DoG[O][S]
        pre_gussian_diff_image = DoG[O][S-1]
        after_gussian_diff_image = DoG[O][S + 1]
        dx = (cur_gussian_diff_image[i, j + 1] - cur_gussian_diff_image[i, j - 1]) / 2
        dy = (cur_gussian_diff_image[i + 1, j] - cur_gussian_diff_image[i - 1, j]) / 2
        ## in the direction of levels or scales
        ds = (after_gussian_diff_image[i, j] - pre_gussian_diff_image[i, j]) / 2

        #TODO: if too slow, make memos to store the first derivative
        dxx = cur_gussian_diff_image[i, j + 1] - 2 * cur_gussian_diff_image[i, j] + cur_gussian_diff_image[i, j - 1]
        dyy = cur_gussian_diff_image[i + 1, j] - 2 * cur_gussian_diff_image[i, j] + cur_gussian_diff_image[i - 1, j]
        dss = after_gussian_diff_image[i, j] - 2 * cur_gussian_diff_image[i, j] + pre_gussian_diff_image[i, j]
        dxy = (cur_gussian_diff_image[i + 1, j + 1] - cur_gussian_diff_image[i + 1, j - 1] - cur_gussian_diff_image[i - 1, j + 1] + cur_gussian_diff_image[i - 1, j - 1]) / 4
        dxs = (after_gussian_diff_image[i, j] - cur_gussian_diff_image[i, j + 1] - pre_gussian_diff_image[i, j] + cur_gussian_diff_image[i, j - 1]) / 4
        dys = (after_gussian_diff_image[i, j] - cur_gussian_diff_image[i + 1, j] - pre_gussian_diff_image[i, j] + cur_gussian_diff_image[i - 1, j]) / 4
        dDD =numpy.array(object=[ [dxx, dxy, dxs],
                                  [dxy, dyy, dys],
                                  [dxs, dys, dss]])
        dD = numpy.array(object=[dx, dy, ds])
        dDDT = numpy.linalg.pinv(dDD)
        x_star = numpy.matmul(dDDT, dD)
        corrected_i = i
        corrected_j = j
        corrected_s = S
        if abs(x_star[0]) > 0.5 or abs(x_star[1]) > 0.5 or abs(x_star[2]) > 0.5:
            #TODO: i->1 or 0 ？ j-> 0 or 1?
            corrected_i += int(x_star[1])
            corrected_j += int(x_star[0])
            corrected_s += int(x_star[2])
            # make sure corrected i, j ,s inside the image
            if corrected_s < 1 or corrected_s > num_features:
                continue
            if corrected_i < 1 or corrected_i >= cur_gussian_diff_image.shape[0] - 1:
                continue
            if corrected_j < 1 or corrected_j >= cur_gussian_diff_image.shape[1] - 1:
                continue
        # check the D(X_start)
        D_x_start = cur_gussian_diff_image[corrected_i, corrected_j] + (dD.dot(numpy.linalg.pinv(x_star)))/2
        if abs(D_x_start) >= contrast_threshold:
            continue
        Tr = dxx + dyy
        det = dxx * dyy - dxy * dxy
        if det <= 0 or  ((Tr*Tr)/det) >= edge_threshold:
            continue
        filtered_and_corrected_extremas.append(O, S, corrected_s, corrected_i, corrected_j) # type: ignore
    logger.info(f"filtered_and_corrected_extremas:{filtered_and_corrected_extremas}")
    row_extremas.clear()
    # generate descriptor
    descriptor = []
    bin = 8
    key_points: typing.List[typing.Tuple[int, int, int, int, int ,float]] = []
    for O, S, corrected_s, i , j in filtered_and_corrected_extremas:
        dxs: typing.List[typing.Union[int, float]] = []
        dys: typing.List[typing.Union[int, float]] = []
        ws = []
        cur_image = guassian_pyramid[O][corrected_s]
        # S is not extract
        cur_sigma = sigmas[O][S]
        G_exp = -(1 / 2 * sigma * sigma)
        histogram = [0.0] * bin
        radius = int(2 * corrected_s)
        for tem_r in range(-radius, radius + 1):
            y = tem_r + i
            if  0 < y < cur_image.shape[0] - 1:
                for tem_r2 in range(-radius , radius + 1):
                    x = tem_r2 + j
                    if 0 < x < cur_image.shape[1] - 1:
                        o_dx = (cur_image[y, x + 1] - cur_image[y, x - 1])
                        o_dy = (cur_image[y - 1, x] - cur_image[y + 1, x])
                        dxs.append(o_dx)
                        dys.append(o_dy)
                        ws.append( math.exp(G_exp * (i**2 + j **2)))
        dxs_ndarr: numpy.ndarray = numpy.array(dxs)
        dys_ndarr: numpy.ndarray = numpy.array(dys)
        ws_ndarr: numpy.ndarray = numpy.array(ws)
        theta = (numpy.arctan2(dys_ndarr, dxs_ndarr) / numpy.pi) * 180
        M = (dxs_ndarr**2 + dys_ndarr ** 2) ** 0.5
        for idx in range(len(dxs)):
            bin_num = int((bin * theta[idx]) / 360)
            bin_num = (bin_num - bin) if bin_num >= bin else (bin_num + bin)
            histogram[bin] += ws_ndarr[idx] * M[idx]
        maxval = max(histogram)
        index_of_max = numpy.argmax(histogram)
        for k in range(bin):
            l = k - 1 if k != 0 else bin -1
            r = k + 1 if k != bin -1 else 0
            if histogram[k] > histogram[l] and histogram[k] > histogram[r] and histogram[k] >= maxval:
                cur_bin = k + 0.5 * (histogram[l]-histogram[r]) /(histogram[l] - 2 * histogram[k] + histogram[r])
                cur_bin = bin + cur_bin if cur_bin < 0 else (cur_bin - bin if cur_bin >= bin else cur_bin)
                key_points.append((O, S, corrected_s, i, j,(360.0/bin) * cur_bin))
        # generate Descriptors
        descriptors = []
        for O, S, corrected_s, i , j, cur_bin in key_points:
            








                        
                

















    
    pass

get_key_points_decipher(image=image1)

2023-10-03 18:45:19,795 - __main__ - INFO - origin_images[i].shape:(480, 640), size:10
INFO:__main__:origin_images[i].shape:(480, 640), size:10
2023-10-03 18:45:22,614 - __main__ - INFO - origin_images[i].shape:(480, 640), size:10
INFO:__main__:origin_images[i].shape:(480, 640), size:10
2023-10-03 18:45:25,464 - __main__ - INFO - origin_images[i].shape:(480, 640), size:10
INFO:__main__:origin_images[i].shape:(480, 640), size:10
2023-10-03 18:45:28,236 - __main__ - INFO - origin_images[i].shape:(480, 640), size:10
INFO:__main__:origin_images[i].shape:(480, 640), size:10
2023-10-03 18:45:30,950 - __main__ - INFO - origin_images[i].shape:(480, 640), size:10
INFO:__main__:origin_images[i].shape:(480, 640), size:10
2023-10-03 18:45:33,639 - __main__ - INFO - origin_images[i].shape:(480, 640), size:10
INFO:__main__:origin_images[i].shape:(480, 640), size:10
2023-10-03 18:45:36,352 - __main__ - INFO - origin_images[i].shape:(480, 640), size:10
INFO:__main__:origin_images[i].shape:(480, 640), 